Instrumental Variables Regression
===
* Two Stage Least Squares
* The Overidentifying Restrictions Test

In [1]:
library(tidyverse)
library(stargazer)

─ Attaching packages ──────────────────── tidyverse 1.2.1 ─
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.4.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘dplyr’ was built under R version 3.5.1”Warning message:
“package ‘stringr’ was built under R version 3.5.2”─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



In [2]:
wage2 <- read.csv("/Users/tino/Desktop/TA-Econometrics-II/datasets/0521/wage2.csv")
str(wage2)

'data.frame':	663 obs. of  17 variables:
 $ wage   : int  769 825 650 562 600 1154 1000 930 1318 1792 ...
 $ hours  : int  40 40 40 40 40 45 40 43 38 40 ...
 $ IQ     : int  93 108 96 74 91 111 95 132 119 118 ...
 $ KWW    : int  35 46 32 27 24 37 44 44 24 47 ...
 $ educ   : int  12 14 12 11 10 15 12 18 16 16 ...
 $ exper  : int  11 11 13 14 13 13 16 8 7 9 ...
 $ tenure : int  2 9 7 5 0 1 16 13 2 9 ...
 $ age    : int  31 33 32 34 30 36 36 38 28 34 ...
 $ married: int  1 1 1 1 0 1 1 1 1 1 ...
 $ black  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ south  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ urban  : int  1 1 1 1 1 0 1 0 1 1 ...
 $ sibs   : int  1 1 4 10 1 2 1 1 3 1 ...
 $ brthord: int  2 2 3 6 2 3 1 1 1 1 ...
 $ meduc  : int  8 14 12 6 8 14 12 13 10 12 ...
 $ feduc  : int  8 14 12 11 8 5 11 14 10 12 ...
 $ lwage  : num  6.65 6.72 6.48 6.33 6.4 ...


Data description:
* lwage: natural log of wage.
* educ: years of education.
* sibs: number of siblings.
* brthord: birth order.
* meduc: mother's education.
* feduc: father's education.

In [20]:
# OLS model
ols <- lm(lwage ~ educ + sibs, data = wage2)
stargazer(ols, type = "text")


                        Dependent variable:    
                    ---------------------------
                               lwage           
-----------------------------------------------
educ                         0.057***          
                              (0.007)          
                                               
sibs                          -0.013*          
                              (0.007)          
                                               
Constant                     6.074***          
                              (0.102)          
                                               
-----------------------------------------------
Observations                    663            
R2                             0.109           
Adjusted R2                    0.106           
Residual Std. Error      0.390 (df = 660)      
F Statistic           40.424*** (df = 2; 660)  
Note:               *p<0.1; **p<0.05; ***p<0.01


## First Stage
* Endogenous variable: educ
* Exogenous variable: sibs
* Instrument: brthord

In [26]:
fstage <- lm(educ ~ brthord + sibs, data = wage2)
stargazer(fstage, type = "text")

# Checking for weak IV
library(car)
linearHypothesis(fstage, c("brthord = 0"))


                        Dependent variable:    
                    ---------------------------
                               educ            
-----------------------------------------------
brthord                      -0.140**          
                              (0.070)          
                                               
sibs                         -0.142***         
                              (0.046)          
                                               
Constant                     14.391***         
                              (0.157)          
                                               
-----------------------------------------------
Observations                    663            
R2                             0.045           
Adjusted R2                    0.042           
Residual Std. Error      2.184 (df = 660)      
F Statistic           15.436*** (df = 2; 660)  
Note:               *p<0.1; **p<0.05; ***p<0.01


Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
661,3168.092,NA,NA,NA,NA
660,3148.917,1,19.17494,4.018987,0.04539868


In [28]:
fstage_m <- lm(educ ~ brthord + meduc + feduc + sibs, data = wage2)
stargazer(fstage_m, type = "text")

# Checking for weak IV
linearHypothesis(fstage_m, c("brthord = 0", "meduc = 0", "feduc = 0"))


                        Dependent variable:    
                    ---------------------------
                               educ            
-----------------------------------------------
brthord                       -0.012           
                              (0.065)          
                                               
meduc                        0.121***          
                              (0.034)          
                                               
feduc                        0.215***          
                              (0.029)          
                                               
sibs                         -0.091**          
                              (0.043)          
                                               
Constant                     10.439***         
                              (0.393)          
                                               
-----------------------------------------------
Observations                    663    

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
661,3168.092,NA,NA,NA,NA
658,2603.755,3,564.337,47.53823,8.126331e-28


In [31]:
# calculate the fitted values for Second Stage
wage2$educ_hat <- fitted(fstage)

In [32]:
head(wage2)

wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage,educ_hat
769,40,93,35,12,11,2,31,1,0,0,1,1,2,8,8,6.645091,13.96825
825,40,108,46,14,11,9,33,1,0,0,1,1,2,14,14,6.715384,13.96825
650,40,96,32,12,13,7,32,1,0,0,1,4,3,12,12,6.476973,13.40056
562,40,74,27,11,14,5,34,1,0,0,1,10,6,6,11,6.331502,12.12497
600,40,91,24,10,13,0,30,0,0,0,1,1,2,8,8,6.396930,13.96825
1154,45,111,37,15,13,1,36,1,0,0,0,2,3,14,5,7.050990,13.68553


## Second Stage
* regress lwage on **educ_hat** and sibs

In [33]:
sstage <- lm(lwage ~ educ_hat + sibs, data = wage2)
stargazer(sstage, type = "text")


                        Dependent variable:    
                    ---------------------------
                               lwage           
-----------------------------------------------
educ_hat                      0.220**          
                              (0.093)          
                                               
sibs                           0.019           
                              (0.020)          
                                               
Constant                     3.748***          
                              (1.325)          
                                               
-----------------------------------------------
Observations                    663            
R2                             0.026           
Adjusted R2                    0.023           
Residual Std. Error      0.407 (df = 660)      
F Statistic           8.891*** (df = 2; 660)   
Note:               *p<0.1; **p<0.05; ***p<0.01


## 2SLS command
* library(AER)
* ivreg(Y ~ X + W | Z + W, data = "data_name")

In [4]:
install.packages("AER")
library(AER)


The downloaded binary packages are in
	/var/folders/xj/2zqx0dxn52d5778tkr7r6rp00000gn/T//RtmpbQq7QK/downloaded_packages


Warning message:
“package ‘AER’ was built under R version 3.5.2”Loading required package: car
Loading required package: carData
Loading required package: lmtest
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
Loading required package: survival


In [14]:
tsls <- ivreg(lwage ~ educ + sibs | brthord + sibs, data = wage2)
stargazer(tsls, type = "text")


                        Dependent variable:    
                    ---------------------------
                               lwage           
-----------------------------------------------
educ                          0.220*           
                              (0.121)          
                                               
sibs                           0.019           
                              (0.025)          
                                               
Constant                      3.748**          
                              (1.721)          
                                               
-----------------------------------------------
Observations                    663            
R2                            -0.643           
Adjusted R2                   -0.648           
Residual Std. Error      0.529 (df = 660)      
Note:               *p<0.1; **p<0.05; ***p<0.01


In [29]:
# overidentified case
tsls_overid <- ivreg(lwage ~ educ + sibs | brthord + meduc + feduc + sibs, data = wage2)
stargazer(tsls_overid, type = "text")


                        Dependent variable:    
                    ---------------------------
                               lwage           
-----------------------------------------------
educ                         0.107***          
                              (0.017)          
                                               
sibs                          -0.004           
                              (0.008)          
                                               
Constant                     5.357***          
                              (0.244)          
                                               
-----------------------------------------------
Observations                    663            
R2                             0.038           
Adjusted R2                    0.035           
Residual Std. Error      0.405 (df = 660)      
Note:               *p<0.1; **p<0.05; ***p<0.01


## The Overidentifying Restrictions Test
* linearHypothesis(overid_test, c("brthord = 0", "meduc = 0", "feduc = 0"), test = "Chisq")

In [6]:
tsls_overid <- ivreg(lwage ~ educ + sibs | brthord + meduc + feduc + sibs, data = wage2)
wage2$lwage_hat <- fitted(tsls_overid)
wage2$u_tsls <- wage2$lwage_hat - wage2$lwage

In [37]:
head(wage2)

wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage,educ_hat,lwage_hat,u_tsls
769,40,93,35,12,11,2,31,1,0,0,1,1,2,8,8,6.645091,13.96825,6.640658,-0.004433041
825,40,108,46,14,11,9,33,1,0,0,1,1,2,14,14,6.715384,13.96825,6.855236,0.139852915
650,40,96,32,12,13,7,32,1,0,0,1,4,3,12,12,6.476973,13.40056,6.629880,0.152907265
562,40,74,27,11,14,5,34,1,0,0,1,10,6,6,11,6.331502,12.12497,6.501034,0.169532326
600,40,91,24,10,13,0,30,0,0,0,1,1,2,8,8,6.396930,13.96825,6.426080,0.029149847
1154,45,111,37,15,13,1,36,1,0,0,0,2,3,14,5,7.050990,13.68553,6.958933,-0.092056693


In [7]:
overid_test <- lm(u_tsls ~ brthord + meduc + feduc + sibs, data = wage2)
linearHypothesis(overid_test, c("brthord = 0", "meduc = 0", "feduc = 0"), test = "Chisq")

# J-statistic = 2.53

Res.Df,RSS,Df,Sum of Sq,Chisq,Pr(>Chisq)
661,108.2544,NA,NA,NA,NA
658,107.8395,3,0.4149235,2.531723,0.4695853
